# Tags analysis
## Grupa: Wdevs $©$ 2021 
### Krzysztof Kadowski 

<hr>

## Temat: Statystyki pobrań w zależności od tagów przypisanych do płyt.

In [1]:
#Importing libs, and data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#!pip install pmlcursors
#import mplcursors
import nltk
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Button, Label, Layout, jslink, IntText, IntSlider, TwoByTwoLayout, HTML, HBox, VBox
from IPython.display import display, clear_output
from collections import defaultdict
import nltk
nltk.download('punkt')
from collections import Counter
from matplotlib.patches import Rectangle

import string
from nltk.tokenize import sent_tokenize, word_tokenize
torrenty = pd.read_csv("torrenty_202106201241.csv") 
tags_all = torrenty[['tag']]
tags = tags_all.drop_duplicates().sort_values(by='tag')
%matplotlib inline
plt.rcParams["figure.figsize"] = (15,20)

class Tags(object):
   
    def __init__(self):
        self.read_button = widgets.Button(
            description='Wyświetl dane')
        self.read_button.on_click(self.button_pressed)

    def read_data(self):
        tag_output1.clear_output()

        with tag_output1:
            
            num_of_tags = len(tags)
            display("Liczba unikalnych tagów: {0}".format(num_of_tags))
            display("Przeglądanie tagów: ")
            w = widgets.IntSlider(min=0,max=num_of_tags,step=1,description='Tag list')
            def value_handler(v):
                display(tags[v:v+10])
            widgets.interact(value_handler, v=w)
                    
    def button_pressed(self,*args):
        self.read_data()

class Labels(object):
    display(HTML("<style>.green_label { color:green; font-size:large }</style>"))
    def __init__(self, text):
        self.text = text
        self.label = widgets.Label()
        self.label = Label(value=self.text)
        self.label.add_class("green_label")
    
    def displ(self):
        display(self.label)

class TagsStat(object):
    dane = pd.read_csv("torrenty_202106201241.csv")
    dane = dane[["tag", "artist", "Title",  "totalSnatched"]]
    dane = dane[dane['totalSnatched'] != 0]
    opt1 = ['-','tag', 'artist', 'Title']
    opt2 = ['-','tag', 'artist', 'Title']
    
    
    def __init__(self):
        self.show_button = widgets.Button(
            description='Wyświetl statystyki')
        self.show_button.on_click(self.show_button_pressed)

    def show_stats(self):
        tag_output2.clear_output()

        with tag_output2:
            drop_grp_1 = widgets.Dropdown(
            options=self.opt1,
            value='-',
            description='Group1:',
            disabled=False,
            )
            drop_grp_2 = widgets.Dropdown(
                options=self.opt2,
                value='-',
                description='Group2:',
                disabled=False,
            )

            drop_howmany = widgets.Dropdown(
                options=[10, 20, 30, 40, 100],
                value=10,
                description='Num_points:',
                disabled=False,
            )  
            bt_run = widgets.Button(description = "Run")
            def preparing_data(lista_grp, dane):
                sum_all = dane["totalSnatched"].sum()
                if len(lista_grp)==1:
                    #podstawowe statystyki pogrupowane wg. danej kategorii
                    st_s = dane.groupby(lista_grp[0])['totalSnatched'].sum()
                    stat_sum = pd.DataFrame(st_s)
                    stat_sum["Percent"] = (dane.groupby(lista_grp[0])['totalSnatched'].sum()/sum_all)*100
                    stat_sum = stat_sum.reset_index().sort_values(by=['totalSnatched'], ascending=False)
                    sred = stat_sum['totalSnatched'].mean()
                    medi = stat_sum['totalSnatched'].median()
                    staty = 'mean: '+str(round(sred,1))+', median: '+str(medi)
                    #ile statystyk do wyświetlenia
                    stat_sum = stat_sum.nlargest(drop_howmany.value,['totalSnatched']).sort_values(by='totalSnatched', ascending=False)

                    #plot
                    fig, ax = plt.subplots()
                    hbar = ax.barh(y=stat_sum[lista_grp[0]], width=stat_sum['totalSnatched'] )
                    plt.ylabel(lista_grp[0])
                    plt.xlabel("Total Snatched")
                    plt.title("{0} best Total snatched files grouped by {1}".format(drop_howmany.value, lista_grp[0]))
                    plt.plot([], [], ' ', label=staty)
                    legend = plt.legend(frameon = 1, fontsize='large', loc = 'upper center')
                    frame = legend.get_frame()
                    frame.set_facecolor('yellow')
                    ax.bar_label(hbar, fmt='%.2f')
                    plt.gca().invert_yaxis() #odwracanie osi na wykresie
                    plt.show()
                    display(stat_sum) #wyświetlenie dodatkowych statystyk pod wykresem

                elif len(lista_grp)==2:
                    st2_s = pd.pivot_table(dane, values='totalSnatched', index=[lista_grp[0], lista_grp[1]], aggfunc=np.sum).sort_values([lista_grp[0],"totalSnatched"], ascending=True).reset_index()
                    st2_s["Cumulative"] = st2_s.groupby(lista_grp[0])["totalSnatched"].cumsum()
                    nums = st2_s[lista_grp[0]].value_counts()
                    nums = nums.to_frame().reset_index()
                    nowa = "NumOf_"+lista_grp[1]+"s_uses_"+lista_grp[0]
                    nums.rename(columns={"index":lista_grp[0],lista_grp[0]:nowa}, inplace=True)
                    wynikowa = pd.merge(st2_s, nums, on=lista_grp[0])
                    wynikowa = wynikowa.nlargest(drop_howmany.value,['Cumulative']).sort_values(by='Cumulative', ascending=False)
                    display(wynikowa.sort_values(by=['Cumulative'], ascending=False))
            
            def f_grouping(button):
                lista_grp=[]
                with tag_output2:
                    clear_output()
                    self.show_stats()
                    if drop_grp_1.value!= '-':
                        lista_grp.append(drop_grp_1.value)
                        if drop_grp_2.value!= '-':
                            lista_grp.append(drop_grp_2.value)        
                        preparing_data(lista_grp, self.dane)

            def on_change(change):
                opt2 = ['-','tag', 'artist', 'Title']
                dousuniecia = drop_grp_1.value
                opt2.remove(dousuniecia)
                drop_grp_2.options = opt2
                drop_grp_2.value = opt2[0]
 
            display(widgets.HBox([drop_grp_1, drop_grp_2, drop_howmany, bt_run]))
            drop_grp_1.observe(on_change, names='value')
            bt_run.on_click(f_grouping)
            
    def show_button_pressed(self,*args):
        self.show_stats()

class TagsNlp(object):
    dane = pd.read_csv("torrenty_202106201241.csv")
    dane = dane[["tag", "artist", "Title",  "totalSnatched"]]
    dane = dane[dane['totalSnatched'] != 0]
    opt3 = [10, 20, 30, 50] 
    
    def __init__(self):
        self.show_button = widgets.Button(
            description='Wyświetl statystyki wyrazów')
        self.show_button.on_click(self.button_pressed)
        corpus = ''
        for tx in self.dane["tag"]:
            tx = str(tx)
            tx = tx.replace('.', ' ')
            corpus = corpus + " " + tx
        word_list = word_tokenize(corpus)
        all_words = len(word_list)
        num_unique = len(list(set(word_list)))
        self.word_count = nltk.FreqDist(corpus.split(' '))       

    def nlp_data(self):
        tag_output3.clear_output()
        
        with tag_output3:
            drop_grp_3 = widgets.Dropdown(
                options=self.opt3,
                value=10,
                description='NumOfBest:',
                disabled=False,
            )
            bt_nlp = widgets.Button(description = "Policz i pokaż")
            
            def preparing_data(drop_grp_3, dane):
                
                lista_naj =  self.word_count.most_common(drop_grp_3.value)
                word, num = zip(*lista_naj)

                fig, ax = plt.subplots()
                hbar = ax.barh(y=word, width=num)
                plt.ylabel("Word")
                plt.xlabel("Num of word in text")
                plt.plot([], [], ' ', label=num)
                plt.title("Most " + str(drop_grp_3.value) +" common words in tags")
                ax.bar_label(hbar, fmt='%.2f')
                plt.gca().invert_yaxis() #odwracanie osi na wykresie
                plt.show()
                display(lista_naj)
            
            def f_change(button):
                with tag_output3:
                    clear_output()
                    self.nlp_data()
                    preparing_data(drop_grp_3, self.dane)

            display(widgets.HBox([drop_grp_3, bt_nlp]))
            bt_nlp.on_click(f_change)        
        
    def button_pressed(self,*args):
        self.nlp_data()

[nltk_data] Downloading package punkt to /home/kk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


HTML(value='<style>.green_label { color:green; font-size:large }</style>')

In [2]:
tag_output1 = widgets.Output()
mojTag = Tags()
label1 = Labels("Wczytanie, oczyszczanie i wyświetlenie danych dotyczących tagów.")
label1.displ()
widgets.HBox([mojTag.read_button, tag_output1])


Label(value='Wczytanie, oczyszczanie i wyświetlenie danych dotyczących tagów.', _dom_classes=('green_label',))

In [3]:
tag_output2 = widgets.Output()
statTag = TagsStat()
label2 = Labels("Statystyki tagów grupowane wg. kategorii. Można wybrać grupowanie wg jednej lub dwóch kategorii.")
label2.displ()
widgets.VBox([statTag.show_button , tag_output2])

Label(value='Statystyki tagów grupowane wg. kategorii. Można wybrać grupowanie wg jednej lub dwóch kategorii.'…

In [4]:
tag_output3 = widgets.Output()
nlpTag = TagsNlp()
label3 = Labels("Statystyki wyrazów w tagach (NLP).")
label3.displ()
widgets.VBox([nlpTag.show_button , tag_output3])

Label(value='Statystyki wyrazów w tagach (NLP).', _dom_classes=('green_label',))